In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import chi2_contingency
import joblib


In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
import pandas as pd
import numpy as np
# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

filepath = ("C:/Users/User/Downloads/Student data.xlsx")

student_data = pd.read_excel(filepath)

In [4]:
def df_info(df):
    info = pd.DataFrame({'Type':df.dtypes})
    isnull = pd.DataFrame({'# IsNull':df.isnull().sum()})
    notnull = pd.DataFrame({'# NotNull' : df.count()})
    total = pd.DataFrame({'# Total': df.isnull().sum() + df.count()})
    percent_missing = pd.DataFrame({'% Missing': (df.isnull().sum() / df.shape[0]) *100})
    nunique = pd.DataFrame({'# Unique': df.nunique()})
    percent_unique = pd.DataFrame({'% Unique': (df.nunique() / df.shape[0]) *100})
    df_summary = pd.concat([info,isnull,notnull,total,nunique,percent_unique,percent_missing], axis=1)

    print('Shape of dataframe:')
    print(df.shape)

    return df_summary

# Function to summarize column info
def column_info(df, columns):
    for column in columns:
        print(f'Column: {column}')
        print(f'Percentage missing values: {(df[column].isnull().sum()/df.shape[0])*100}')
        print(f'Number of unique values: {df[column].nunique()}')
        print(f'Unique values: {df[column].unique()}')
        print("")

    return None

In [5]:
student_data.columns = [
    "First_Term_Gpa", "Second_Term_Gpa", "First_Language", "Funding",
    "School", "FastTrack", "Coop", "Residency", "Gender",
    "Prev_Education", "Age_Group", "High_School_Avg_Mark",
    "Math_Score", "English_Grade", "FirstYearPersistence"
]

In [6]:
student_data.replace('?', pd.NA, inplace=True)

In [7]:
numeric_columns = [
    "First_Term_Gpa", "Second_Term_Gpa", "High_School_Avg_Mark",
    "Math_Score", "English_Grade"
]

In [8]:
student_data[numeric_columns] = student_data[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [9]:
categorical_columns = ["First_Language", "Funding", "School", "FastTrack",
                       "Coop", "Residency", "Gender", "Prev_Education", "Age_Group","FirstYearPersistence"]
student_data[categorical_columns] = student_data[categorical_columns].astype(str)
categorical_df = student_data[categorical_columns].copy()

In [10]:
df_info(student_data)

Shape of dataframe:
(1437, 15)


,Type,# IsNull,# NotNull,# Total,# Unique,% Unique,% Missing
First_Term_Gpa,float64,17,1420,1437,692,48.155880,1.183020
Second_Term_Gpa,float64,160,1277,1437,717,49.895616,11.134308
First_Language,object,0,1437,1437,4,0.278358,0.000000
Funding,object,0,1437,1437,6,0.417537,0.000000
School,object,0,1437,1437,1,0.069589,0.000000
FastTrack,object,0,1437,1437,2,0.139179,0.000000
Coop,object,0,1437,1437,2,0.139179,0.000000
Residency,object,0,1437,1437,2,0.139179,0.000000
Gender,object,0,1437,1437,3,0.208768,0.000000
Prev_Education,object,0,1437,1437,4,0.278358,0.000000


In [11]:
#looking at the info I will drop the Highschool Average marks column and the Math_Score columns

In [12]:
student_data = student_data.drop(columns=["High_School_Avg_Mark", "Math_Score"])

In [13]:
from scipy.stats import chi2_contingency
SIGNIFICANCE_LEVEL = 0.05
chi_dict={}

for column in categorical_df.drop(columns=['FirstYearPersistence']).columns:
    contingency_table = pd.crosstab(categorical_df[column], categorical_df['FirstYearPersistence'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    chi_dict[column] = chi2

    print("Chi-square test for " + f"{column}:")
    print(f"Chi2 value: {chi2}")
    print(f"P-value: {p}")

    if p < SIGNIFICANCE_LEVEL:
        print("Significant association with the target variable")
    else:
        print("No significant association with the target variable")

    print("")

Chi-square test for First_Language:
Chi2 value: 56.98358667042711
P-value: 2.5906965128672277e-12
Significant association with the target variable

Chi-square test for Funding:
Chi2 value: 74.0893149825314
P-value: 1.440872478834121e-14
Significant association with the target variable

Chi-square test for School:
Chi2 value: 0.0
P-value: 1.0
No significant association with the target variable

Chi-square test for FastTrack:
Chi2 value: 46.042644424460654
P-value: 1.1570675494372764e-11
Significant association with the target variable

Chi-square test for Coop:
Chi2 value: 0.0026450695201630566
P-value: 0.9589826891295842
No significant association with the target variable

Chi-square test for Residency:
Chi2 value: 63.05274103667974
P-value: 2.0124512999713454e-15
Significant association with the target variable

Chi-square test for Gender:
Chi2 value: 11.153195042143285
P-value: 0.0037854234555005544
Significant association with the target variable

Chi-square test for Prev_Education:

In [14]:
#these are the columns needed to drop because of the chisquared test Coop,School done on categorical columns

In [15]:
student_data = student_data.drop(columns=["Coop", "School"])

In [16]:
df_info(student_data)

Shape of dataframe:
(1437, 11)


,Type,# IsNull,# NotNull,# Total,# Unique,% Unique,% Missing
First_Term_Gpa,float64,17,1420,1437,692,48.155880,1.183020
Second_Term_Gpa,float64,160,1277,1437,717,49.895616,11.134308
First_Language,object,0,1437,1437,4,0.278358,0.000000
Funding,object,0,1437,1437,6,0.417537,0.000000
FastTrack,object,0,1437,1437,2,0.139179,0.000000
Residency,object,0,1437,1437,2,0.139179,0.000000
Gender,object,0,1437,1437,3,0.208768,0.000000
Prev_Education,object,0,1437,1437,4,0.278358,0.000000
Age_Group,object,0,1437,1437,9,0.626305,0.000000
English_Grade,float64,45,1392,1437,8,0.556715,3.131524


In [17]:
X = student_data.drop(columns=["FirstYearPersistence"])
y = student_data["FirstYearPersistence"]

numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [18]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

In [19]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

In [20]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


In [21]:
X_processed = preprocessor.fit_transform(X)
print(type(X_processed))
print(X_processed.shape)

<class 'numpy.ndarray'>
(1437, 33)


In [22]:
joblib.dump(preprocessor, "preprocessor.pkl")

['preprocessor.pkl']

In [23]:
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_processed, y)
print(y.size)
print(y_resampled.size)
print(y.value_counts())
print(y_resampled.value_counts())


sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in sss.split(X_resampled, y_resampled):
    X_train_resampled = X_resampled[train_idx]
    X_test_resampled = X_resampled[test_idx]
    y_train_resampled = y_resampled[train_idx].astype(int)
    y_test_resampled = y_resampled[test_idx].astype(int)

print("ok")

1437
2276
FirstYearPersistence
1    1138
0     299
Name: count, dtype: int64
FirstYearPersistence
1    1138
0    1138
Name: count, dtype: int64
ok


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout


model = Sequential([
    Dense(64, activation="relu", input_dim=X_train_resampled.shape[1]),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dropout(0.2),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(
    X_train_resampled, y_train_resampled,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6422 - loss: 0.6326 - val_accuracy: 0.7060 - val_loss: 0.5552
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7807 - loss: 0.4944 - val_accuracy: 0.7198 - val_loss: 0.5321
Epoch 3/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7819 - loss: 0.4606 - val_accuracy: 0.7390 - val_loss: 0.5181
Epoch 4/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7874 - loss: 0.4449 - val_accuracy: 0.7500 - val_loss: 0.5035
Epoch 5/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7785 - loss: 0.4662 - val_accuracy: 0.7418 - val_loss: 0.4933
Epoch 6/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8065 - loss: 0.4119 - val_accuracy: 0.7445 - val_loss: 0.4909
Epoch 7/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7954 - loss: 0.4375 - val_accuracy: 0.7308 - val_loss: 0.4918
Epoch 8/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8107 - loss: 0.4235 - val_accuracy: 0.7445 - v

In [25]:
loss, accuracy = model.evaluate(X_test_resampled, y_test_resampled, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

y_pred_probs = model.predict(X_test_resampled)
y_pred = (y_pred_probs > 0.5).astype(int) # Convert probabilities to class labels

from sklearn.metrics import classification_report
print(classification_report(y_test_resampled, y_pred))

Test Loss: 0.4343
Test Accuracy: 0.8333
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

           0       0.84      0.83      0.83       228
           1       0.83      0.84      0.83       228

    accuracy                           0.83       456
   macro avg       0.83      0.83      0.83       456
weighted avg       0.83      0.83      0.83       456



In [26]:
import joblib

joblib.dump(model, "nn.pkl")



['nn.pkl']

In [28]:
student_data

,First_Term_Gpa,Second_Term_Gpa,First_Language,Funding,FastTrack,Residency,Gender,Prev_Education,Age_Group,English_Grade,FirstYearPersistence
0,0.000000,0.000000,1,2,2,1,2,1,1,7.0,1
1,2.500000,2.000000,3,4,1,2,2,1,3,7.0,1
2,4.250000,3.923077,1,1,2,1,1,2,3,9.0,1
3,3.020833,2.321429,3,4,1,2,2,2,3,8.0,1
4,4.275000,4.326923,1,2,1,1,1,2,3,9.0,1
...,...,...,...,...,...,...,...,...,...,...,...
1432,1.413043,0.000000,<NA>,4,2,2,2,1,1,10.0,0
1433,4.360000,4.074074,1,2,1,1,1,2,4,10.0,1
1434,2.020833,2.450000,1,2,2,1,2,1,1,9.0,1
1435,4.021739,4.357143,<NA>,4,2,2,1,1,2,8.0,1
